# Gaussian Elimination Method

Solving a system of linear equations using the Gaussian elimination method involves transforming the coefficient matrix of the system into an upper triangular matrix, followed by solving the system using back substitution.

This is one of the most fundamental methods for solving systems of linear equations, especially in cases where the number of unknowns is different from the number of equations.

In [ ]:
from sympy import Matrix, symbols, Eq, latex
from IPython.display import display, Markdown
import sympy as sp

class SymbolicMatrix:
    def __init__(self, matrix, column_labels=None, free_terms=None, display_mode="table"):
        self.matrix = Matrix(matrix)
        self.operations = []
        self.display_mode = display_mode

        if column_labels:
            if len(column_labels) != self.matrix.cols:
                raise ValueError("The number of column labels must be equal to the number of columns in the matrix.")
            self.column_labels = column_labels
        else:
            self.column_labels = [f"x{i+1}" for i in range(self.matrix.cols)]

        if free_terms:
            if len(free_terms) != self.matrix.rows:
                raise ValueError("The number of free terms must be equal to the number of rows in the matrix.")
            self.free_terms = free_terms
        else:
            self.free_terms = [0 for _ in range(self.matrix.rows)]

        display(Markdown("**Initial equations:**"))
        self.display_matrix()

    def set_display_mode(self, mode):
        if mode not in ["table", "equations"]:
            raise ValueError("Display mode must be 'table' or 'equations'.")
        self.display_mode = mode

    def display_matrix(self):
        if self.display_mode == "table":
            self._display_table()
        elif self.display_mode == "equations":
            self._display_equations()

    def _display_table(self):
        matrix_with_free_terms = self.matrix.row_join(Matrix(self.free_terms))
        headers = self.column_labels + ["Free term"]
        display(Markdown(self._matrix_to_markdown(matrix_with_free_terms, headers)))

    def _matrix_to_markdown(self, matrix, headers):
        markdown_str = "| " + " | ".join(headers) + " |\n"
        markdown_str += "| " + " | ".join(["---"] * len(headers)) + " |\n"
        for i in range(matrix.rows):
            row = [str(matrix[i, j]) for j in range(matrix.cols)]
            markdown_str += "| " + " | ".join(row) + " |\n"
        return markdown_str

    def _display_equations(self):
        equations = []
        for i in range(self.matrix.rows):
            terms = []
            for j in range(self.matrix.cols):
                coef = self.matrix[i, j]
                var = symbols(self.column_labels[j])

                # Check if the coefficient is positive, negative, or zero
                if coef > 0:
                    if j == 0:
                        terms.append(f"{latex(coef)} {latex(var)}")
                    else:
                        terms.append(f"+ {latex(coef)} {latex(var)}")
                elif coef < 0:
                    terms.append(f"- {latex(abs(coef))} {latex(var)}")
                elif coef == 0:
                    if j == 0:
                        terms.append(f"0 {latex(var)}")
                    else:
                        terms.append(f"+0 {latex(var)}")

            lhs = " ".join(terms)
            rhs = latex(self.free_terms[i])

            equation = f"{lhs} = {rhs}"
            equations.append(equation)

        display(Markdown("System of equations:"))
        display(Markdown(f"$$\\begin{{aligned}} {self._equations_to_latex(equations)} \\end{{aligned}}$$"))

    def _equations_to_latex(self, equations):
        return " \\\\ ".join(equations)

    def add_row(self, target_row, source_row, coefficient):
        target_idx = self._validate_row_number(target_row)
        source_idx = self._validate_row_number(source_row)

        # Operation on the matrix
        self.matrix.row_op(target_idx, lambda v, j: v + coefficient * self.matrix[source_idx, j])

        # Operation on the free terms
        self.free_terms[target_idx] += coefficient * self.free_terms[source_idx]

        operation_str = f"r{target_row} = r{target_row} + {coefficient}*r{source_row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def multiply_row(self, row, coefficient):
        row_idx = self._validate_row_number(row)

        # Operation on the matrix
        self.matrix.row_op(row_idx, lambda v, _: coefficient * v)

        # Operation on the free terms
        self.free_terms[row_idx] *= coefficient

        operation_str = f"r{row} = {coefficient}*r{row}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def swap_rows(self, row1, row2):
        row1_idx = self._validate_row_number(row1)
        row2_idx = self._validate_row_number(row2)

        self.matrix.row_swap(row1_idx, row2_idx)
        self.free_terms[row1_idx], self.free_terms[row2_idx] = self.free_terms[row2_idx], self.free_terms[row1_idx]

        operation_str = f"Swap r{row1} <-> r{row2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def swap_columns(self, col1, col2):
        col1_idx = self._validate_col_number(col1)
        col2_idx = self._validate_col_number(col2)

        self.matrix.col_swap(col1_idx, col2_idx)
        self.column_labels[col1_idx], self.column_labels[col2_idx] = self.column_labels[col2_idx], self.column_labels[col1_idx]

        operation_str = f"Swap col{col1} <-> col{col2}"
        self.operations.append(operation_str)
        display(Markdown(f"**Operation:** {operation_str}"))
        self.display_matrix()

    def _validate_row_number(self, row):
        if not isinstance(row, int):
            raise TypeError("Row number must be an integer.")
        if row < 1 or row > self.matrix.rows:
            raise IndexError(f"Row number must be between 1 and {self.matrix.rows}.")
        return row - 1

    def _validate_col_number(self, col):
        if not isinstance(col, int):
            raise TypeError("Column number must be an integer.")
        if col < 1 or col > self.matrix.cols:
            raise IndexError(f"Column number must be between 1 and {self.matrix.cols}.")
        return col - 1

    def print_operations(self):
        display(Markdown("**Performed operations:**"))
        for op in self.operations:
            print(op)

# Example usage:
m = SymbolicMatrix([[1,2,3], [4,5,6], [7,8,12]], column_labels=["x1", "x2", "x3"], free_terms=[10, 11, 18], display_mode="equations")


**Initial equations:**

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 4 x_{1} + 5 x_{2} + 6 x_{3} = 11 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(2, 1, -4)

**Operation:** r2 = r2 + -4*r1

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 7 x_{1} + 8 x_{2} + 12 x_{3} = 18 \end{aligned}$$

In [ ]:
m.add_row(3, 1, -7)

**Operation:** r3 = r3 + -7*r1

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} - 6 x_{2} - 9 x_{3} = -52 \end{aligned}$$

In [ ]:
m.add_row(3, 2, -2)

**Operation:** r3 = r3 + -2*r2

System of equations:

$$\begin{aligned} 1 x_{1} + 2 x_{2} + 3 x_{3} = 10 \\ 0 x_{1} - 3 x_{2} - 6 x_{3} = -29 \\ 0 x_{1} +0 x_{2} + 3 x_{3} = 6 \end{aligned}$$

In [ ]:
# We have 3 equations, so we end the operation
# Define symbols
x1, x2, x3 = symbols('x1 x2 x3')
# Define equations
eq1 = Eq(x1 + 2*x2 + 3*x3, 10)
eq2 = Eq(-3*x2 - 6*x3, -29)
eq3 = Eq(3*x3, 6)

In [ ]:
sol_x3 = sp.solve(eq3, x3)[0]
sol_x3

2

In [ ]:
# Substitute solution x3 into equation 2
eq2 = eq2.subs(x3, sol_x3)
# Solve equation 2
sol_x2 = sp.solve(eq2, x2)[0]
sol_x2

17/3

In [ ]:
# Substitute solutions x2 and x3 into equation 1
eq1 = eq1.subs({x2: sol_x2, x3: sol_x3})
# Solve equation 1
sol_x1 = sp.solve(eq1, x1)[0]
sol_x1

-22/3

---

## Exercises for Students

$$
\left\{\begin{array}{c}{x+2 y-2 z=4} \\ {2 x+y+z=0} \\ {3 x+2 y+z=1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y+z-t=2} \\ {2 x+y+z=3} \\ {-x+z-t=0} \\ {3 x+2 y-z+2 t=-1}\end{array}\right.\quad
\left\{\begin{array}{c}{x+y-z-t=0} \\ {2 x+3 y-2 z+t=4} \\ {3 x+5 z=0} \\ {-x+y-3 z+2 t=3}\end{array}\right.
$$

1) Given system:
$$
\begin{cases}
x + 2y - 2z = 4 \\
2x + y + z = 0 \\
3x + 2y + z = 1
\end{cases}
$$

**Steps:**

1. **Write the Equations:**
$$
\begin{aligned}
1. & \quad x + 2y - 2z = 4 \quad \text{(Equation 1)} \\
2. & \quad 2x + y + z = 0 \quad \text{(Equation 2)} \\
3. & \quad 3x + 2y + z = 1 \quad \text{(Equation 3)}
\end{aligned}
$$

2. **Use Equation 1 to Find \(x\):**
$$
x = 4 - 2y + 2z
$$

3. **Substitute This Expression into Other Equations to Find \(y\) and \(z\):**

   - **Substituting into Equation 2:**
$$
2(4 - 2y + 2z) + y + z = 0
$$
$$
8 - 4y + 4z + y + z = 0
$$
$$
-3y + 5z + 8 = 0 \quad \Rightarrow \quad 3y - 5z = 8 \quad \text{(Equation 4)}
$$

   - **Substituting into Equation 3:**
$$
3(4 - 2y + 2z) + 2y + z = 1
$$
$$
12 - 6y + 6z + 2y + z = 1
$$
$$
-4y + 7z + 12 = 1 \quad \Rightarrow \quad 4y - 7z = 11 \quad \text{(Equation 5)}
$$

4. **Solve the New Equations:**
$$
\begin{cases}
3y - 5z = 8 \quad \text{(Equation 4)} \\
4y - 7z = 11 \quad \text{(Equation 5)}
\end{cases}
$$

5. **Find \(y\) from Equation 4:**
$$
y = \frac{8 + 5z}{3}
$$

6. **Substituting into Equation 5:**
$$
4\left(\frac{8 + 5z}{3}\right) - 7z = 11
$$
$$
\frac{32 + 20z}{3} - 7z = 11
$$
$$
32 + 20z - 21z = 33 \quad \text{(Multiply both sides by 3)}
$$
$$
-z = 1 \quad \Rightarrow \quad z = -1
$$

7. **Using \(z\) to Find \(y\):**
$$
y = \frac{8 + 5(-1)}{3} = \frac{8 - 5}{3} = \frac{3}{3} = 1
$$

8. **Using \(y\) and \(z\) to Find \(x\):**
$$
x = 4 - 2(1) + 2(-1) = 4 - 2 - 2 = 0
$$

**Solution:**
$$
(x, y, z) = (0, 1, -1)
$$


2)
Given system:
$$
\begin{cases}
x + y + z - t = 2 \\
2x + y + z = 3 \\
-x + z - t = 0 \\
3x + 2y - z + 2t = -1
\end{cases}
$$

**Steps:**

1. **Write the Equations:**
$$
\begin{aligned}
1. & \quad x + y + z - t = 2 \quad \text{(Equation 1)} \\
2. & \quad 2x + y + z = 3 \quad \text{(Equation 2)} \\
3. & \quad -x + z - t = 0 \quad \text{(Equation 3)} \\
4. & \quad 3x + 2y - z + 2t = -1 \quad \text{(Equation 4)}
\end{aligned}
$$

2. **Use Equation 1 to Find \(x\):**
$$
x = 2 - y - z + t
$$

3. **Substitute This Expression into Other Equations to Find \(y\), \(z\), and \(t\):**

   - **Substituting into Equation 2:**
$$
2(2 - y - z + t) + y + z = 3
$$
$$
4 - 2y - 2z + 2t + y + z = 3
$$
$$
-y - z + 2t = -1 \quad \Rightarrow \quad y + z - 2t = 1 \quad \text{(Equation 5)}
$$

   - **Substituting into Equation 3:**
$$
-(2 - y - z + t) + z - t = 0
$$
$$
-2 + y + z - t + z - t = 0
$$
$$
y + 2z - 2t = 2 \quad \text{(Equation 6)}
$$

4. **Solve the New Equations:**
$$
\begin{cases}
y + z - 2t = 1 \quad \text{(Equation 5)} \\
y + 2z - 2t = 2 \quad \text{(Equation 6)}
\end{cases}
$$

5. **Subtract Equation 5 from Equation 6 to Find \(z\):**
$$
(y + 2z - 2t) - (y + z - 2t) = 2 - 1
$$
$$
z = 1
$$

6. **Using \(z = 1\) in Equation 5 to Find \(y\):**
$$
y + 1 - 2t = 1
$$
$$
y = 2t
$$

7. **Using \(y\) and \(z\) in Equation 1 to Find \(x\):**
$$
x = 2 - (2t) - 1 + t
$$
$$
x = 1 - t
$$

**Solution:**
$$
(x, y, z, t) = (1 - t, 2t, 1, t)
$$


3) Given system:
$$
\begin{cases}
x + y - z - t = 0 \\
2x + 3y - 2z + t = 4 \\
3x + 5z = 0 \\
-x + y - 3z + 2t = 3
\end{cases}
$$

**Steps:**

1. **Write the Equations:**
$$
\begin{aligned}
1. & \quad x + y - z - t = 0 \quad \text{(Equation 1)} \\
2. & \quad 2x + 3y - 2z + t = 4 \quad \text{(Equation 2)} \\
3. & \quad 3x + 5z = 0 \quad \text{(Equation 3)} \\
4. & \quad -x + y - 3z + 2t = 3 \quad \text{(Equation 4)}
\end{aligned}
$$

2. **Use Equation 1 to Find \(x\):**
$$
x = -y + z + t
$$

3. **Substitute This Expression into Other Equations to Find \(y\), \(z\), and \(t\):**

   - **Substituting into Equation 2:**
$$
2(-y + z + t) + 3y - 2z + t = 4
$$
$$
-2y + 2z + 2t + 3y - 2z + t = 4
$$
$$
y + 3t = 4 \quad \Rightarrow \quad y = 4 - 3t \quad \text{(Equation 5)}
$$

   - **Substituting into Equation 3:**
$$
3(-y + z + t) + 5z = 0
$$
$$
-3y + 3z + 3t + 5z = 0
$$
$$
-3y + 8z + 3t = 0 \quad \Rightarrow \quad 3y = 8z + 3t \quad \text{(Equation 6)}
$$

4. **Substitute \(y = 4 - 3t\) into Equation 6 to Find \(z\):**
$$
3(4 - 3t) = 8z + 3t
$$
$$
12 - 9t = 8z + 3t
$$
$$
12 = 8z + 12t
$$

